In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers.string import StrOutputParser
import sys
sys.path.append('../src/')
from chain.model import llm

SYSTEM_TEMPLATE = """You are an AI assistant specialized in generating database queries for a movie recommendation system."""

USER_TEMPLATE = """<Instruction>
Please create a three types of query to retrieve the necessary information from the movie data to answer the User's query.
Three types of queries:
    1. SQL query for structured "movie" table.
    2. Vector DB query for movie plot summary.
    3. Vector DB query for movie reviews.
</Instruction>

<SQL_query>
    <movie_table_description>
    The "movie" table contains data on various movies, including metadata such as director, screenwriter, plot, and ratings.
    </movie_table_description>
    <Description_of_Each_Column_in_the_Movie_Table>
    title: The title of the movie. (TEXT)
    director: A list of the movie's directors. (LIST)
    screenwriter: A list of the movie's screenwriters. (LIST)
    plot: A short summary of the movie's plot. (TEXT)
    rating: The average rating given to the movie. Range 0.0 ~ 5.0 (FLOAT)
    rating_count: The total number of ratings the movie has received. (FLOAT)
    actors: A list of the movie's main actors. (LIST)
    genres: A list of the genres. (LIST)
    countries: A list of the countries where the movie was produced. (LIST)
    audience: Cumulative audience (FLOAT)
    running_time: The running time of the movie in minutes. (FLOAT)
    adult: A flag indicating whether the movie is for adults only (1.0 for true, 0.0 for false). (FLOAT)
    </Description_of_Each_Column_in_the_Movie_Table>
</SQL_query>

<User_query>
{query}
</User_query>

<Response_Format>
{{
    "sql_query": SQL query for structured "movie" table,
    "vector_db_query_plot": Vector DB query in korean for movie plot summary,
    "vector_db_query_reviews": Vector DB query in korean for movie reviews
}}
</Response_Format>
"""


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", SYSTEM_TEMPLATE),
        ("user", USER_TEMPLATE),
    ]
)


generate_sql_query_chain = prompt | llm | StrOutputParser()


In [3]:
res = generate_sql_query_chain.invoke("평점이 높은 영화 중에서 미국 대학생들의 스토리가 담긴 영화를 추천해줘")

In [4]:
print(res)

{
    "sql_query": "SELECT title, director, screenwriter, plot, rating, rating_count FROM movie WHERE rating >= 4.0 AND plot LIKE '%대학생%';",
    "vector_db_query_plot": "미국 대학생들의 이야기가 담긴 영화의 줄거리 요약을 검색해 주세요.",
    "vector_db_query_reviews": "미국 대학생들의 스토리가 포함된 영화에 대한 리뷰를 검색해 주세요."
}
